In [1]:
import pandas as pd
import numpy as np

from parental_care.config import BLD
from parental_care.config import SRC

In [2]:
def filter_nested_dict(original_dict, keys_to_remove):
    return {
        key: [value for value in values if value not in keys_to_remove.get(key, [])]
        if key in keys_to_remove
        else values
        for key, values in original_dict.items()
    }

In [3]:
def process_wave(wave_number, data_modules):
    wave_data = {}

    for module in data_modules.keys():
        print(f"module: {module}")
        module_file = (
            SRC / f"data/sharew{wave_number}/sharew{wave_number}_rel8-0-0_{module}.dta"
        )

        # Read and filter
        if module == "re" and wave_number == 7:
            wave_module = pd.read_stata(module_file, convert_categoricals=False)
            wave_module = wave_module[wave_module["country"] == 12]

            #lookup = {
            #    f"re{var[2:]}": f"sl{var[2:]}"
            #    for var in data_modules["re"]
            #    if var.startswith("sl")
            #}
            lookup = {
                f"{var[3:]}": f"{var}"
                for var in data_modules["re"]
                if var.startswith("sl")
            }
        else:
            wave_module = pd.read_stata(module_file, convert_categoricals=True)
            wave_module = wave_module[wave_module["country"] == "Germany"]

            lookup = {
                "sp009_1sp": "sp009_1",
                "sp009_2sp": "sp009_2",
                "sp009_3sp": "sp009_3",
                "sp019d2sp": "sp019d2",
                "sp019d3sp": "sp019d3",
                "sp019d4sp": "sp019d4",
                "sp019d5sp": "sp019d5",
                "sp019d6sp": "sp019d6",
                "sp019d7sp": "sp019d7",
            }

        # Rename columns using the dictionary
        wave_module.rename(columns=lookup, inplace=True)

        module_vars = ["mergeid"] + data_modules[module]

        # Select columns
        wave_module = wave_module[module_vars]

        wave_data[module] = wave_module

        print(wave_module.shape)

    add_wealth_data = "gv_imputations" in data_modules
    merged_data = wave_data["cv_r"]

    data_modules.pop("cv_r")
    data_modules.pop("gv_imputations", None)

    for module_key in data_modules.keys():
        merged_data = merged_data.merge(
            wave_data[module_key], on="mergeid", how="outer"
        )

    if add_wealth_data:
        merged_data = merged_data.merge(
            wave_data["gv_imputations"], on="mergeid", how="left"
        )

    merged_data["wave"] = wave_number

    return merged_data

In [4]:
all_variables = {
    "cv_r": [
        "int_year",
        "int_month",
        "gender",
        "mobirth",
        "yrbirth",
        "age_int",
        "hhsize",
    ],
    "dn": [
        "dn002_",
        "dn003_",
        "dn010_",
        "dn041_",
        "dn009_",
        "dn014_",
        "dn015_",
        "dn016_",
        "dn026_1",
        "dn026_2",
        "dn033_1",
        "dn033_2",
        "dn028_1",
        "dn028_2",
        "dn030_1",
        "dn030_2",
        "dn127_1",
        "dn127_2",
        "dn032_1",
        "dn032_2",
        "dn012d1",
        "dn012d2",
        "dn012d3",
        "dn012d4",
        "dn012d5",
        "dn012d6",
        "dn012d7",
        "dn012d8",
        "dn012d9",
        "dn012d10",
        "dn012d11",
        "dn012d12",
        "dn012d13",
        "dn012d14",
        "dn012d15",
        "dn012d16",
        "dn012d17",
        "dn012d18",
        "dn012d19",
        "dn012d20",
        #"dn012d95",
        "dn012dno",
        "dn012dot",
    ],
    "ep": [
        "ep005_",
        "ep002_",
        "ep328_",
        "ep329_",
        "ep213_1",
        #"ep213_2",
        #"ep213_3",
        #"ep213_4",
        #"ep213_5",
        #"ep213_6",
        #"ep213_7",
        #"ep213_8",
        #"ep213_9",
        #"ep213_10",
        #"ep213_11",
        #"ep213_12",
        #"ep213_13",
        #"ep213_14",
        #"ep213_15",
        #"ep213_16",
    ],
    "sp": [
        "sp008_",
        "sp018_",
        "sp009_1",
        "sp009_2",
        "sp009_3",
        "sp010d1_1",
        "sp010d1_2",
        "sp010d1_3",
        "sp011_1",
        "sp011_2",
        "sp011_3",
        "sp019d2",
        "sp019d3",
        "sp019d4",
        "sp019d5",
        "sp019d6",
        "sp019d7",
    ],
    "gv_isced": ["isced1997_r"],
    #"gv_imputations": [
    #    "hnetw"
    #],  # household net worth = total gross financial assets + total real assets - total libailities
    "ch": ["ch001_"],
}

In [5]:
keys_to_remove_wave1 = {
    "dn": [
        "dn041_",
        "dn127_1",
        "dn127_2",
        "dn012d15",
        "dn012d16",
        "dn012d17",
        "dn012d18",
        "dn012d19",
        "dn012d20",
    ],
    "ep": [
        "ep328_",
        "ep329_",
        "ep213_12",
        "ep213_13",
        "ep213_14",
        "ep213_15",
        "ep213_16",
    ],
}

keys_to_remove_wave2 = {
    "dn": [
        "dn127_1",
        "dn127_2",
        "dn012d15",
        "dn012d16",
        "dn012d17",
        "dn012d18",
        "dn012d19",
        "dn012d20",
    ],
}


keys_to_remove_wave4 = {
    "dn": [
        "dn127_1",
        "dn127_2",
        "dn012d14",
        "dn012d15",
        "dn012d16",
        "dn012d17",
        "dn012d18",
        "dn012d19",
        "dn012d20",
    ],
    "sp": [
        "sp010d1_1",
        "sp010d1_2",
        "sp010d1_3",
    ],
}


keys_to_remove_wave5 = {
    "dn": [
        "dn127_1",
        "dn127_2",
        "dn012d20",
        "dn012dno",
    ],
    "sp": [
        "sp010d1_1",
        "sp010d1_2",
        "sp010d1_3",
    ],
}

keys_to_remove_wave6 = {
    "dn": [
        "dn012dno",
    ],
    "ep": ["ep213_14", "ep213_15", "ep213_16"],
}

keys_to_remove_wave7 = {
    "dn": [
        "dn012dno",
    ],
    "ep": ["ep213_14", "ep213_15", "ep213_16"],
    "re": [
        "sl_re018_1",
        "sl_re018_2",
        "sl_re018_3",
        "sl_re018_4",
        "sl_re018_5",
        "sl_re018_6",
        "sl_re018_7",
        "sl_re020_1",
        "sl_re020_2",
        "sl_re020_3",
        "sl_re020_4",
        "sl_re020_5",
        "sl_re020_6",
        "sl_re020_7",
    ],
}

keys_to_remove_wave8 = {
    "dn": [
        "dn012dno",
    ],
    "ep": ["ep213_14", "ep213_15", "ep213_16"],
}

In [6]:
# Retrospective waves
re_vars = (
    [f"sl_re011_{i}" for i in range(1, 21)]
    + [f"sl_re016_{i}" for i in range(1, 21)]
    + [f"sl_re026_{i}" for i in range(1, 21)]
    + [f"sl_re018_{i}" for i in range(1, 8)]
    + [f"sl_re020_{i}" for i in range(1, 8)]
)

# Data modules for wave 3
variables_wave3 = {
    "cv_r": [
        "int_year",
        "int_month",
        "gender",
        "mobirth",
        "yrbirth",
        "age_int",
        "hhsize",
    ],
    "re": re_vars,
}

# Separate modules for partly retrospective wave 7
variables_wave7 = filter_nested_dict(all_variables | {"re": re_vars}, keys_to_remove_wave7) 

In [7]:
variables_wave1 = filter_nested_dict(all_variables, keys_to_remove_wave1)
variables_wave2 = filter_nested_dict(all_variables, keys_to_remove_wave2)
variables_wave4 = filter_nested_dict(all_variables, keys_to_remove_wave4)
variables_wave5 = filter_nested_dict(all_variables, keys_to_remove_wave5)
variables_wave6 = filter_nested_dict(all_variables, keys_to_remove_wave6)
variables_wave8 = filter_nested_dict(all_variables, keys_to_remove_wave8)

In [8]:
wave1 = process_wave(wave_number=1, data_modules=variables_wave1)
wave2 = process_wave(wave_number=2, data_modules=variables_wave2)
wave3 = process_wave(wave_number=3, data_modules=variables_wave3)
wave4 = process_wave(wave_number=4, data_modules=variables_wave4)
wave5 = process_wave(wave_number=5, data_modules=variables_wave5)
wave6 = process_wave(wave_number=6, data_modules=variables_wave6)
wave7 = process_wave(wave_number=7, data_modules=variables_wave7)
wave8 = process_wave(wave_number=8, data_modules=variables_wave8)

module: cv_r
(3920, 8)
module: dn
(2995, 34)
module: ep
(2995, 4)
module: sp
(2995, 18)
module: gv_isced
(2995, 2)
module: ch
(2995, 2)
module: cv_r
(3504, 8)
module: dn
(2628, 35)
module: ep
(2628, 6)
module: sp
(2628, 18)
module: gv_isced
(2628, 2)
module: ch
(2628, 2)
module: cv_r
(2501, 8)
module: re
(1918, 75)
module: cv_r
(2146, 8)
module: dn
(1619, 34)
module: ep
(1619, 6)
module: sp
(1619, 15)
module: gv_isced
(1619, 2)
module: ch
(1619, 2)
module: cv_r
(7674, 8)
module: dn
(5750, 39)
module: ep
(5750, 6)
module: sp
(5750, 15)
module: gv_isced
(5750, 2)
module: ch
(5750, 2)
module: cv_r
(5787, 8)
module: dn
(4411, 42)
module: ep
(4411, 6)
module: sp
(4411, 18)
module: gv_isced
(4411, 2)
module: ch
(4411, 2)
module: cv_r
(4928, 8)
module: dn
(3820, 42)
module: ep
(3820, 6)
module: sp
(3820, 18)
module: gv_isced
(3820, 2)
module: ch
(3820, 2)
module: re
(3820, 61)
module: cv_r
(4202, 8)
module: dn
(2878, 42)
module: ep
(2878, 6)
module: sp
(2878, 18)
module: gv_isced
(2878, 2)
mo

In [102]:
waves_list = [wave1, wave2, wave3, wave4, wave5, wave6, wave7, wave8]

# Drop all nan rows
for i, df in enumerate(waves_list):
    waves_list[i] = df.dropna(how='all', axis=0, inplace=False)
    #waves_list[i] = df.dropna(axis=1, how='all')

In [103]:
def merge_wave_datasets(wave_datasets):
    # Combine the data frames in wave_datasets into one data frame
    #     combined_data = pd.concat(wave_datasets)
    combined_data = pd.concat(wave_datasets, axis=0, ignore_index=True)

    # Filter out rows where the 'int_year' column is not equal to -9
    #combined_data = combined_data[combined_data["int_year"] != -9]
    combined_data = combined_data[combined_data["int_year"] != "Not applicable"]

    # Sort the data frame by 'mergeid' and 'int_year'
    combined_data = combined_data.sort_values(by=["mergeid", "int_year"])

    return combined_data

In [104]:
data = merge_wave_datasets(waves_list)

/tmp/ipykernel_18560/3547024282.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_data = pd.concat(wave_datasets, axis=0, ignore_index=True)


In [105]:
data.shape

(26593, 149)

In [106]:
sum(pd.crosstab(data["int_year"], columns='Count')["Count"])

26590

In [107]:
len(list(data))

149

In [108]:
nan_dropped = [
    "mergeid",
    "int_year",
    "int_month",
    "gender",
    "mobirth",
    "yrbirth",
    "age_int",
    "hhsize",
    "dn002_",
    "dn003_",
    "dn010_",
    "dn009_",
    "dn014_",
    "dn015_",
    "dn016_",
    "dn026_1",
    "dn026_2",
    "dn033_1",
    "dn033_2",
    "dn028_1",
    "dn028_2",
    "dn030_1",
    "dn030_2",
    "dn032_1",
    "dn032_2",
    "dn012d1",
    "dn012d2",
    "dn012d3",
    "dn012d4",
    "dn012d5",
    "dn012d6",
    "dn012d7",
    "dn012d8",
    "dn012d9",
    "dn012d10",
    "dn012d11",
    "dn012d12",
    "dn012d13",
    "dn012d14",
    "dn012dno",
    "dn012dot",
    "ep005_",
    "ep002_",
    "ep213_1",
    "sp008_",
    "sp018_",
    "sp009_1",
    "sp009_2",
    "sp009_3",
    "sp010d1_1",
    "sp010d1_2",
    "sp010d1_3",
    "sp011_1",
    "sp011_2",
    "sp011_3",
    "sp019d2",
    "sp019d3",
    "sp019d4",
    "sp019d5",
    "sp019d6",
    "sp019d7",
    "isced1997_r",
    "ch001_",
    "wave",
    "dn041_",
    "ep328_",
    "ep329_",
    "sl_re011_1",
    "sl_re011_2",
    "sl_re011_3",
    "sl_re011_4",
    "sl_re011_5",
    "sl_re011_6",
    "sl_re011_7",
    "sl_re011_8",
    "sl_re011_9",
    "sl_re011_10",
    "sl_re011_11",
    "sl_re016_1",
    "sl_re016_2",
    "sl_re016_3",
    "sl_re016_4",
    "sl_re016_5",
    "sl_re016_6",
    "sl_re016_7",
    "sl_re016_8",
    "sl_re016_9",
    "sl_re016_10",
    "sl_re016_11",
    "sl_re026_1",
    "sl_re026_2",
    "sl_re026_3",
    "sl_re026_4",
    "sl_re026_5",
    "sl_re026_6",
    "sl_re026_7",
    "sl_re026_8",
    "sl_re026_9",
    "sl_re026_10",
    "sl_re026_11",
    "sl_re018_1",
    "sl_re018_2",
    "sl_re018_3",
    "sl_re018_4",
    "sl_re018_5",
    "sl_re018_6",
    "sl_re018_7",
    "sl_re020_1",
    "sl_re020_2",
    "sl_re020_3",
    "sl_re020_4",
    "sl_re020_5",
    "sl_re020_6",
    "sl_re020_7",
    "dn012d15",
    "dn012d16",
    "dn012d17",
    "dn012d18",
    "dn012d19",
    "dn127_1",
    "dn127_2",
    "dn012d20",
    "sl_re011_12",
    "sl_re011_13",
    "sl_re016_12",
    "sl_re016_13",
    "sl_re026_12",
    "sl_re026_13",
]

In [111]:
set_empty_cols = {
    "sl_re011_14",
    "sl_re011_15",
    "sl_re011_16",
    "sl_re011_17",
    "sl_re011_18",
    "sl_re011_19",
    "sl_re011_20",
    "sl_re016_14",
    "sl_re016_15",
    "sl_re016_16",
    "sl_re016_17",
    "sl_re016_18",
    "sl_re016_19",
    "sl_re016_20",
    "sl_re026_14",
    "sl_re026_15",
    "sl_re026_16",
    "sl_re026_17",
    "sl_re026_18",
    "sl_re026_19",
    "sl_re026_20",
}

In [99]:
r_list

['mergeid',
 'int_year',
 'int_month',
 'gender',
 'mobirth',
 'yrbirth',
 'age_int',
 'hhsize',
 'dn002_',
 'dn003_',
 'dn010_',
 'dn009_',
 'dn014_',
 'dn015_',
 'dn016_',
 'dn026_1',
 'dn026_2',
 'dn033_1',
 'dn033_2',
 'dn028_1',
 'dn028_2',
 'dn030_1',
 'dn030_2',
 'dn032_1',
 'dn032_2',
 'dn012d1',
 'dn012d2',
 'dn012d3',
 'dn012d4',
 'dn012d5',
 'dn012d6',
 'dn012d7',
 'dn012d8',
 'dn012d9',
 'dn012d10',
 'dn012d11',
 'dn012d12',
 'dn012d13',
 'dn012d14',
 'dn012d15',
 'dn012d16',
 'dn012d17',
 'dn012d18',
 'dn012d19',
 'dn012d20',
 'dn012dno',
 'dn012dot',
 'ep005_',
 'ep002_',
 'ep213_1',
 'sp008_',
 'sp018_',
 'sp009_1',
 'sp009_2',
 'sp009_3',
 'sp010d1_1',
 'sp010d1_2',
 'sp010d1_3',
 'sp011_1',
 'sp011_2',
 'sp011_3',
 'sp019d2',
 'sp019d3',
 'sp019d4',
 'sp019d5',
 'sp019d6',
 'sp019d7',
 'isced1997_r',
 'ch001_',
 'wave',
 'dn041_',
 'ep328_',
 'ep329_',
 'sl_re011_1',
 'sl_re011_2',
 'sl_re011_3',
 'sl_re011_4',
 'sl_re011_5',
 'sl_re011_6',
 'sl_re011_7',
 'sl_re011_8'

# 2) Data preparation

## Discrete state variables

###### states which are interpolated
- labor market experience (0, 30)
- years in retirement (0, 6)
- years in intensive care (0, 5)
- father age (70, 90)
- mother age (70, 90)
###### states which are not interpolated
- individuals’ type (1, 2)
- father died last period (0, 1)
- mother died last period (0, 1)
- father alive (0, 1)
- mother alive (0, 1)
- health of father (1, 2, 3)
- health of mother (1, 2, 3)
- existence of siblings (0, 1) --> **change to existence of sister?**
- parents live close by (0, 1)
- married (0, 1)
- education (low, high)

In [114]:
dat = data.copy()